In [3]:
import cv2
import os
import numpy as np

def mask_background_keep_largest(
    input_folder: str,
    output_folder: str,
    bg_color: tuple,
    tolerance: int = 10
):
    """
    Process each image in `input_folder` to create a binary mask by removing a uniform background
    and then keep only the largest connected white region in the mask, removing any small lines or patches.
    
    Args:
        input_folder (str): Path to the folder containing the input images.
        output_folder (str): Path where the generated masks will be saved.
        bg_color (tuple): The background color in BGR format (e.g., (59, 59, 59)).
        tolerance (int): Tolerance for color difference. Pixels with a total difference <= this
                         value are treated as background.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, filename)
            img = cv2.imread(image_path)
            if img is None:
                print(f"Warning: Could not load image {image_path}")
                continue

            # Create an array matching the background color
            bg_array = np.full(img.shape, bg_color, dtype=np.uint8)

            # Compute the absolute difference and sum across color channels
            diff = cv2.absdiff(img, bg_array)
            diff_sum = np.sum(diff, axis=2)

            # Create a binary mask: > tolerance => foreground, else background
            mask = np.uint8((diff_sum > tolerance).astype(np.uint8)) * 255

            # Optionally, a morphological opening to remove tiny noise
            kernel = np.ones((3, 3), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

            # Identify connected components
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)

            # If there's only background (num_labels < 2), skip
            if num_labels < 2:
                # If no foreground region found, just save the empty mask
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, mask)
                print(f"No foreground found in {filename}; saved empty mask.")
                continue

            # Keep only the largest connected component (label != 0 is background)
            largest_label = 1
            largest_area = stats[1, cv2.CC_STAT_AREA]

            for label_id in range(2, num_labels):
                area = stats[label_id, cv2.CC_STAT_AREA]
                if area > largest_area:
                    largest_area = area
                    largest_label = label_id

            # Create a mask containing only the largest component
            largest_mask = np.zeros_like(mask)
            largest_mask[labels == largest_label] = 255

            # Save the refined mask
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, largest_mask)
            print(f"Processed and saved mask for: {filename}")

if __name__ == "__main__":
    # Example usage
    input_dir = r"D:\ASCC_parts_extended\workspace\dist_far\train"  # Folder with input images
    output_dir = r"D:\ASCC_parts_extended\workspace\dist_far\masks"  # Folder to save masks

    # Background color in BGR (59,59,59 in your example)
    bg_color = (59, 59, 59)

    # Tolerance for background difference
    tolerance = 10

    mask_background_keep_largest(input_dir, output_dir, bg_color, tolerance)


Processed and saved mask for: 0001.png
Processed and saved mask for: 0002.png
Processed and saved mask for: 0003.png
Processed and saved mask for: 0004.png
Processed and saved mask for: 0005.png
Processed and saved mask for: 0006.png
Processed and saved mask for: 0007.png
Processed and saved mask for: 0008.png
Processed and saved mask for: 0009.png
Processed and saved mask for: 0010.png
Processed and saved mask for: 0011.png
Processed and saved mask for: 0012.png
Processed and saved mask for: 0013.png
Processed and saved mask for: 0014.png
Processed and saved mask for: 0015.png
Processed and saved mask for: 0016.png
Processed and saved mask for: 0017.png
Processed and saved mask for: 0018.png
Processed and saved mask for: 0019.png
Processed and saved mask for: 0020.png
Processed and saved mask for: 0021.png
Processed and saved mask for: 0022.png
Processed and saved mask for: 0023.png
Processed and saved mask for: 0024.png
Processed and saved mask for: 0025.png
Processed and saved mask 

In [4]:
import json
import os

# Specify your input JSON file and the output JSON filename
input_json = "transforms.json"  # Change this to the path of your JSON file
output_json = "transforms_with_masks.json"

# Folder where mask images are stored (relative or absolute)
mask_folder = "masks"  # e.g., if your masks are in a folder called "masks"

# Load the original JSON
with open(input_json, "r") as f:
    data = json.load(f)

# Process each frame in the JSON file
for frame in data.get("frames", []):
    # Get the base filename from the file_path (e.g., "0001.png")
    image_filename = os.path.basename(frame["file_path"])
    
    # Create the corresponding mask path; adjust extension if needed.
    # For example, if your mask files are JPEG, you could replace the extension:
    # mask_filename = os.path.splitext(image_filename)[0] + ".jpeg"
    # Here we simply assume the mask has the same filename.
    mask_filename = image_filename

    # Create the relative path to the mask image
    frame["mask_path"] = os.path.join(mask_folder, mask_filename)

# (Optional) Save the updated JSON into a new file
with open(output_json, "w") as f:
    json.dump(data, f, indent=4)

print(f"Updated JSON has been saved to {output_json}")


Updated JSON has been saved to transforms_with_masks.json
